In [ ]:
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.dates as mdates
import os
from matplotlib.dates import DateFormatter


In [ ]:
os.chdir('data')

In [ ]:

dataset = netCDF4.Dataset('./precipitation/era5_hourly/precip.nc', 'r')
time = dataset.variables['time'][:]
time_units = dataset.variables['time'].units
time_calendar = dataset.variables['time'].calendar
dates = netCDF4.num2date(time, units=time_units, calendar=time_calendar)
dates = [datetime(d.year, d.month, d.day, d.hour, d.minute, d.second) for d in dates]
precipitation = dataset.variables['precip'][:]  
# Access the latitude and longitude variables
latitudes = dataset.variables['latitude'][:]
longitudes = dataset.variables['longitude'][:]

# Load the inventory
gdf = gpd.read_file('./inventory/landslides_liguria.shp')
gdf['utc_date'] = pd.to_datetime(gdf['utc_date'], format="%d/%m/%Y %H:%M")


pixel_events = {}

# Loop through each event in the inventory
for index, row in gdf.iterrows():
    event_date = row['utc_date']
    location = row['geometry']
    if isinstance(location, Point):
        lat, lon = location.y, location.x
        
        lat_index = np.argmin(np.abs(latitudes - lat))
        lon_index = np.argmin(np.abs(longitudes - lon))

        pixel_key = (lat_index, lon_index)
        if pixel_key not in pixel_events:
            pixel_events[pixel_key] = []
        pixel_events[pixel_key].append(event_date)


# Function to filter rainfall events
def filter_rainfall_events(precipitation, dates, threshold=0.2, min_duration=2, max_gap=24):
    events = []
    event = []
    event_dates = []
    gap_counter = 0  # Counter for hours below the threshold

    for i in range(len(dates)):
        if precipitation[i] > threshold:
            event.append(precipitation[i])
            event_dates.append(dates[i])
            gap_counter = 0  # Reset the gap counter when rainfall is above the threshold
        else:
            if len(event) > 0:
                gap_counter += 1
                if gap_counter > max_gap:
                    # End the current event if the gap exceeds the allowed maximum
                    if len(event) > 0:
                        events.append((event, event_dates))
                    event = []
                    event_dates = []
                    gap_counter = 0  # Reset the gap counter

    # Add the last event if it exists
    if len(event) > 0:
        events.append((event, event_dates))

    # Filter events based on minimum duration
    filtered_events = [(e, d) for e, d in events if len(e) > min_duration]
    return filtered_events


'''
def filter_rainfall_events(precipitation, dates, threshold=0.2, min_duration=2):
    events = []
    event = []
    event_dates = []
    dry_season = range(6, 10)  # June to September
    wet_season = chain(range(10, 13), range(1, 6))  # October to May

    for i in range(len(dates)):
        if precipitation[i] > threshold:
            event.append(precipitation[i])
            event_dates.append(dates[i])
        else:
            if len(event) > 0:
                events.append((event, event_dates))
                event = []
                event_dates = []

    if len(event) > 0:
        events.append((event, event_dates))

    filtered_events = []
    for event, event_dates in events:
        start_month = event_dates[0].month
        end_month = event_dates[-1].month
        if (start_month in dry_season and end_month in dry_season and 
            all((event_dates[i+1] - event_dates[i]).total_seconds() / 3600 < 48 for i in range(len(event_dates) - 1))):
            filtered_events.append((event, event_dates))
        elif (start_month in wet_season and end_month in wet_season and 
              all((event_dates[i+1] - event_dates[i]).total_seconds() / 3600 < 96 for i in range(len(event_dates) - 1))):
            filtered_events.append((event, event_dates))

    #print("Events:", events)
    #print("Filtered Events:", filtered_events)
    return [e for e in filtered_events if len(e[0]) > min_duration]

'''


dataset.close()

from datetime import datetime, timedelta


pixel_dataframes = {}

for index, row in gdf.iterrows():
    event_date = row['utc_date']
    location = row['geometry']
    if isinstance(location, Point):
        lat, lon = location.y, location.x
        
        lat_index = np.argmin(np.abs(latitudes - lat))
        lon_index = np.argmin(np.abs(longitudes - lon))

        pixel_key = (lat_index, lon_index)
        if pixel_key not in pixel_events:
            pixel_events[pixel_key] = []
        pixel_events[pixel_key].append(event_date)
        


for (lat_index, lon_index), gdf_event_dates in pixel_events.items():
    pixel_precipitation = precipitation[:, lat_index, lon_index]
    filtered_events = filter_rainfall_events(pixel_precipitation, dates)
    
    # Create a DataFrame for the pixel
    data = []

    for event, event_dates in filtered_events:
        start_time = event_dates[0]
        end_time = event_dates[-1]
        duration = (end_time - start_time).total_seconds() / 3600  # duration in hours
        rainfall_intensity = np.mean(event)
        cumulative_rf = np.sum(event)

        
        LS_date = None  
        #Landslide_id = None
        for gdf_event_date in gdf_event_dates:
            
            
        
            # Check if the landslide event occurs within the rainfall event or up to 48 hours after
            if start_time <= gdf_event_date <= end_time + timedelta(hours=48):
                LS_date = gdf_event_date.strftime("%Y-%m-%d %H:%M:%S")
                #Landslide_id = gdf_id
                break  
            

        occurrences = 1 if LS_date else 0  # Set occurrences to 1 if LS_date is found, otherwise 0
        data.append([LS_date,  start_time, end_time, duration, rainfall_intensity, cumulative_rf, occurrences, event])  # Add event (rainfall values) as a list
    
    df = pd.DataFrame(data, columns=['LS_date',  'start_time', 'end_time', 'duration', 'rainfall_intensity', 'Cumulative_RF', 'occurrences', 'rainfall_timeseries'])
    pixel_dataframes[(lat_index, lon_index)] = df

total_occurrences = 0
# Print the DataFrames for each pixel
for pixel, df in pixel_dataframes.items():
    print(f"DataFrame for pixel {pixel}:")
    print(df['occurrences'].sum())
    total_occurrences += df['occurrences'].sum()
    #display(df)
print(f"Total occurrences for all pixels: {total_occurrences}")    

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


plots_shown = 0
max_plots_to_show = 10  # Limit the number of plots to display to 10

# Iterate through each pixel's DataFrame
for pixel, df in pixel_dataframes.items():
   
    filtered_df = df[df['occurrences'] == 1]
    
    
    print(f"Rows with occurrences == 1 for pixel {pixel}:")
    display(filtered_df)
    
    # Iterate through the filtered rows
    for index, row in filtered_df.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']
        time_steps = pd.date_range(start=start_time, end=end_time, freq='H')  # One-hour intervals
        
        # Convert LS_date to pandas.Timestamp
        LS_date = pd.to_datetime(row['LS_date'])
        
        # Debugging: Print start_time, end_time, and LS_date
        print(f"Pixel: {pixel}, Start: {start_time}, End: {end_time}, LS_date: {LS_date}")
        
        # Ensure rainfall_timeseries is valid
        rainfall_timeseries = row['rainfall_timeseries']
        if not isinstance(rainfall_timeseries, (list, pd.Series)):
            print(f"Invalid rainfall_timeseries for pixel {pixel}, skipping...")
            continue
        
        # Ensure time_steps and rainfall_timeseries have the same length
        if len(time_steps) != len(rainfall_timeseries):
            print(f"Length mismatch for pixel {pixel}: time_steps ({len(time_steps)}) vs rainfall_timeseries ({len(rainfall_timeseries)}). Adjusting...")
            time_steps = time_steps[:len(rainfall_timeseries)]  # Trim time_steps to match rainfall_timeseries
        
        # Find the index of LS_date in time_steps
        if LS_date in time_steps:
            ls_index = time_steps.get_loc(LS_date)
        else:
            print(f"LS_date ({LS_date}) not found in time_steps for pixel {pixel}, skipping...")
            continue
        
        # Check if all points before LS_date are less than 0.5
        if all(value < 0.5 for value in rainfall_timeseries[:ls_index]):
            print(f"All points before LS_date are less than 0.5 for pixel {pixel}, deleting row...")
            filtered_df.drop(index, inplace=True)  # Delete the row from the DataFrame
            continue  
        
        
        if plots_shown < max_plots_to_show:
            plt.figure(figsize=(10, 5))
            plt.plot(time_steps, rainfall_timeseries, marker='o', label=f"Pixel: {pixel}, Start: {start_time}")
            
            # Add LS_date as a red dashed vertical line (always plot it)
            plt.axvline(x=LS_date, color='red', linestyle='--', label='LS_date')
            
            # Add a note if LS_date is outside the range
            if not (start_time <= LS_date <= end_time):
                print(f"Note: LS_date ({LS_date}) is outside the range of start_time ({start_time}) and end_time ({end_time}).")
            
            plt.title(f"Rainfall Timeseries for Pixel {pixel}")
            plt.xlabel("Time")
            plt.ylabel("Rainfall (mm)")
            plt.xticks(rotation=45)
            plt.legend()
            plt.grid()
            plt.show()
            
            plots_shown += 1  # Increment the plot counter
    
    # Continue processing all rows, even if the plot limit is reached

In [ ]:

def plot_scatter(pixel_dataframes,a):
    for pixel, df in pixel_dataframes.items():
        # Filter data based on occurrences
        df_occurrences_0 = df[df['occurrences'] == 0]
        df_occurrences_above_0 = df[df['occurrences'] > 0]

        
        plt.scatter(df_occurrences_0['duration'], df_occurrences_0[a], color='blue', alpha=0.2, label='Non Occurrences')
        plt.scatter(df_occurrences_above_0['duration'], df_occurrences_above_0[a], color='red', alpha=0.5, label='Occurrences')

        
        plt.title(f'Scatter Plot for {pixel}')
        plt.xlabel('Duration (log scale)')
        plt.ylabel(a)
        plt.legend()
        plt.grid()

        plt.xscale('log')
        plt.yscale('log')
        plt.show()


a = 'rainfall_intensity'
#a= 'Cumulative_RF'
plot_scatter(pixel_dataframes,a)

In [ ]:
#For the whole area 01

import pandas as pd
import matplotlib.pyplot as plt

def plot_combined_scatter(pixel_dataframes, a):
    # Validate and reset index for each DataFrame
    cleaned_dataframes = []
    for pixel, df in pixel_dataframes.items():
        print(f"Processing pixel: {pixel}")
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"Value for pixel '{pixel}' is not a DataFrame.")
        if df.empty:
            print(f"Warning: DataFrame for pixel '{pixel}' is empty and will be skipped.")
            continue
        # Debugging: Print DataFrame shape and index length
        print(f"DataFrame shape: {df.shape}, Index length: {len(df.index)}")
        # Ensure the index matches the data length
        if len(df.index) != len(df):
            print(f"Warning: DataFrame for pixel '{pixel}' has a mismatched index. Resetting index.")
            df = df.reset_index(drop=True)
        # Convert pixel to string and assign it as a column
        cleaned_dataframes.append(df.assign(pixel=str(pixel)))

    # Merge all cleaned DataFrames into one
    if not cleaned_dataframes:
        raise ValueError("No valid DataFrames to plot.")
    combined_df = pd.concat(cleaned_dataframes, ignore_index=True)
    cd_output_file = "Rainfall_Events_2002-2022.xlsx"
    combined_df.to_excel(cd_output_file, index=False)

    # Filter data based on occurrences
    df_occurrences_0 = combined_df[combined_df['occurrences'] == 0]
    df_occurrences_above_0 = combined_df[combined_df['occurrences'] > 0]

    
    plt.scatter(df_occurrences_0['duration'], df_occurrences_0[a], color='blue', alpha=0.2, label='Non Occurrences')
    plt.scatter(df_occurrences_above_0['duration'], df_occurrences_above_0[a], color='red', alpha=0.5, label='Occurrences')

    
    plt.title('Combined Scatter Plot')
    plt.xlabel('Duration (log scale)')
    plt.ylabel(a)
    plt.legend()
    plt.grid()

    
    plt.xscale('log')
    plt.yscale('log')

    
    plt.show()


#a = 'rainfall_intensity'
a = 'Cumulative_RF'
plot_combined_scatter(pixel_dataframes, a)

In [ ]:
#For the whole area 02
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def calculate_rainfall_threshold(df, RF_metric):
    percentiles = np.arange(0, 101, 5)
    thresholds = np.percentile(df[RF_metric], percentiles)

    results = []

    for threshold in thresholds:
        df['Prediction'] = np.where(df[RF_metric] >= threshold, 1, 0)
        
        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(df['occurrences'], df['Prediction']).ravel()
        
        # Calculate TPR and TNR
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        
        # Calculate Youden's Index
        youden_index = tpr + tnr - 1
        
        results.append({
            'Threshold': threshold,
            'TPR': tpr,
            'TNR': tnr,
            'Youden_Index': youden_index
        })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Step 3: Determine the optimal threshold based on Youden's Index
    optimal_threshold = results_df.loc[results_df['Youden_Index'].idxmax()]

    # Output the optimal threshold
    print("Optimal Rainfall Threshold (RT):", optimal_threshold['Threshold'])
    print("True Positive Rate (TPR):", optimal_threshold['TPR'])
    print("True Negative Rate (TNR):", optimal_threshold['TNR'])
    print("Youden's Index:", optimal_threshold['Youden_Index'])
    return results_df

def combine_and_calculate_threshold(pixel_dataframes, RF_metric):
    # Combine all dataframes into one
    combined_df = pd.concat(
        [df.reset_index(drop=True).assign(pixel=str(pixel)) for pixel, df in pixel_dataframes.items()],
        ignore_index=True
    )

    
    if 'occurrences' not in combined_df.columns or RF_metric not in combined_df.columns:
        raise ValueError(f"The combined dataframe must contain 'occurrences' and '{RF_metric}' columns.")

    # Calculate the rainfall threshold
    results_df = calculate_rainfall_threshold(combined_df, RF_metric)
    return results_df


RF_metric = 'Cumulative_RF'
optimal_values = combine_and_calculate_threshold(pixel_dataframes, RF_metric)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix


def calculate_rainfall_threshold(df,RF_metric):
# Step 1: Vary the percentile threshold
    percentiles = np.arange(0, 101, 5)
    thresholds = np.percentile(df[RF_metric], percentiles)

# Step 2: Calculate TPR, TNR, and Youden's Index
    results = []

    for threshold in thresholds:
        df['Prediction'] = np.where(df[RF_metric] >= threshold, 1, 0)
        
        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(df['occurrences'], df['Prediction']).ravel()
        
        # Calculate TPR and TNR
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        
        # Calculate Youden's Index
        youden_index = tpr + tnr - 1
        
        results.append({
            'Threshold': threshold,
            'TPR': tpr,
            'TNR': tnr,
            'Youden_Index': youden_index
        })

# Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Step 3: Determine the optimal threshold based on Youden's Index
    optimal_threshold = results_df.loc[results_df['Youden_Index'].idxmax()]

    # Output the optimal threshold
    #print("Optimal Rainfall Threshold (RT):", optimal_threshold['Threshold'])
    #print("True Positive Rate (TPR):", optimal_threshold['TPR'])
    #print("True Negative Rate (TNR):", optimal_threshold['TNR'])
    #print("Youden's Index:", optimal_threshold['Youden_Index'])
    #return results_df
    return {
        'Optimal_Threshold': optimal_threshold['Threshold'],
        'Optimal_Youden_Index': optimal_threshold['Youden_Index'],
        "True Positive Rate (TPR)": optimal_threshold['TPR'],
        "True Negative Rate (TNR)": optimal_threshold['TNR'],
    }



summary_table = []

for pixel, df in pixel_dataframes.items():
    total_occurrences = df['occurrences'].sum()
    total_observations = df.shape[0]
    occurrence_rate = (total_occurrences / total_observations) * 100
    
    # Calculate optimal threshold and Youden's Index
    optimal_values = calculate_rainfall_threshold(df, 'Cumulative_RF')
    
    # Append results to the summary table
    summary_table.append({
        'Pixel': pixel,
        'Total_Observations': total_observations,
        'Occurrence_Rate (%)': occurrence_rate,
        "True Positive Rate (TPR)": optimal_values['True Positive Rate (TPR)'],
        "True Negative Rate (TPR)": optimal_values['True Negative Rate (TNR)'],
        'Optimal_Youden_Index': optimal_values['Optimal_Youden_Index'],
        'Rainfall_Threshold': optimal_values['Optimal_Threshold']
    })

# Convert summary table to DataFrame and display
summary_df = pd.DataFrame(summary_table)
display(summary_df)
output_file = "Rainfall_Summary.xlsx"
summary_df.to_excel(output_file, index=False)
    

In [ ]:
shapefile_cleaned = gpd.read_file(f'Example_Susceptibility_25_11_2016/Centroids_RF_sampled_ver01.shp')
#shapefile_cleaned['geometry'] = shapefile_cleaned.geometry.buffer(0.000256, cap_style=3)

In [ ]:
from shapely.geometry import Point


point1 = shapefile_cleaned.geometry.iloc[0]
point2 = shapefile_cleaned.geometry.iloc[1]

# Calculate the distance between the two points
distance = point1.distance(point2)

# Use half of the distance as the buffer distance
buffer_distance = distance / 2

# Apply the buffer to each point using the calculated buffer distance
shapefile_cleaned['geometry'] = shapefile_cleaned.geometry.buffer(buffer_distance, cap_style=3)

# Print the updated GeoDataFrame
print(shapefile_cleaned.head())

In [ ]:
#shapefile_cleaned.explore()

In [ ]:
import geopandas as gpd
import pandas as pd
# Load the GeoDataFrame
gdf = gpd.read_file('./inventory/landslides_liguria.shp')
gdf['utc_date'] = pd.to_datetime(gdf['utc_date'], format="%d/%m/%Y %H:%M")
pointfile = gdf

pointfile = pointfile.to_crs(shapefile_cleaned.crs)
overlay_result = gpd.sjoin(shapefile_cleaned, pointfile, how='left', predicate='intersects')
overlay_result['Landslide'] = overlay_result['index_right'].notnull().astype(int)


In [ ]:

landslide_sum = overlay_result['Landslide'].sum()
print(f"Total Landslide overlaps: {landslide_sum}")


In [ ]:
print(overlay_result)

In [ ]:
RF_TH = 

In [ ]:
# Define the LSI ranges based on PL_df
lsi_values = np.arange(0.1, 1.1, 0.1)  
PL_df = pd.DataFrame({'LSI': lsi_values})
print(PL_df)
lsi_ranges = PL_df['LSI'].tolist()
lsi_bins = [0] + lsi_ranges  # Add 0 as the lower bound
print(lsi_bins)

# Filter rows where Landslide column is 1
landslide_rows = overlay_result[overlay_result['Landslide'] == 1]

# Bin the LSI02 values into the defined ranges for filtered rows
landslide_rows['LSI_bin'] = pd.cut(landslide_rows['LSI02'], bins=lsi_bins, right=False)

# Count the number of rows in each bin where Landslide is 1
lsi_landslide_counts = landslide_rows['LSI_bin'].value_counts().sort_index()

# Bin the LSI02 values into the defined ranges
overlay_result['LSI_bin'] = pd.cut(overlay_result['LSI02'], bins=lsi_bins, right=False)

# Count the total number of rows in each bin
lsi_total_counts = overlay_result['LSI_bin'].value_counts().sort_index()

# Combine both counts into a single DataFrame
lsi_counts_df = pd.DataFrame({
    'Landslide_Counts': lsi_landslide_counts,
    'Total_Counts': lsi_total_counts
}).fillna(0)  # Fill NaN values with 0

print(lsi_counts_df)

In [ ]:
#calcultating the change in LSI due to rainfall